# Homework - 2 - Considering Bias in Data

Some Portions of code within this notebook have been developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program, the said code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/).

In [33]:
# 
# These are standard python modules
import json, time, urllib.parse
#
# The 'requests' module is not a standard Python module. You will need to install this with pip/pip3 if you do not already have it
import requests

import os

# The 'pandas' module is not a standard Python module. You will need to install this with pip/pip3 if you do not already have it
import pandas as pd

## Loading and Transforming the Data

In [44]:
# Storing "us_cities_by_state_SEPT.2023.csv" as a pandas dataframe

cities_data = pd.read_csv("us_cities_by_state_SEPT.2023.csv", index_col = False)

In [45]:
cities = cities_data['page_title'].tolist()
urls = cities_data['url'].tolist()

In [46]:
cities_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22157 entries, 0 to 22156
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   state       22157 non-null  object
 1   page_title  22157 non-null  object
 2   url         22157 non-null  object
dtypes: object(3)
memory usage: 519.4+ KB


In [47]:
cities_data['state'].nunique()

48

In [48]:
# Storing "US States by Region - US Census Bureau.xlsx" as a pandas dataframe
 
states_region_data = pd.read_excel("US States by Region - US Census Bureau.xlsx")

# Creating a column called regional_division. Values Format: State (Division)
states_region_data['regional_division'] = states_region_data['REGION'] + ' (' + states_region_data['DIVISION'] + ') '

states_region_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   REGION             50 non-null     object
 1   DIVISION           50 non-null     object
 2   STATE              50 non-null     object
 3   regional_division  50 non-null     object
dtypes: object(4)
memory usage: 1.7+ KB


In [49]:
# Storing "NST-EST2022-POP.xlsx" as a pandas dataframe
pop_2022 = pd.read_excel("NST-EST2022-POP.xlsx")
pop_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Geographic Area               56 non-null     object
 1   April 1, 2020 Estimates Base  56 non-null     int64 
 2   2022                          56 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.4+ KB


In [50]:
# Left Join performed on states_region_data and pop_2022.
# Key for states_region_data is 'STATE'
# Key for pop_2022 is 'Geographic Area'

states_region_pop = pd.merge(states_region_data[['STATE', 'regional_division']], pop_2022[['Geographic Area', 2022]], left_on='STATE', right_on='Geographic Area', how= 'left' )
states_region_pop.rename(columns={2022: 'Population (2022)'}, inplace=True)
states_region_pop.drop(columns=['Geographic Area'], inplace=True)


states_region_pop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   STATE              50 non-null     object
 1   regional_division  50 non-null     object
 2   Population (2022)  50 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.6+ KB


In [51]:
# Left Join cities_data and states_region_pop
# Key for cities_data: 'state'
# Key for states_region_pop: 'STATE'
state_city_data = pd.merge(cities_data, states_region_pop, left_on='state', right_on='STATE', how= 'left')
state_city_data.drop(columns=['STATE'], inplace=True)

#state_city_data.to_csv("final_state_city_data.csv", index=False)

state_city_data.head()

,state,page_title,url,regional_division,Population (2022)
0,Alabama,"Abbeville, Alabama","https://en.wikipedia.org/wiki/Abbeville,_Alabama",South (East South Central),5074296
1,Alabama,"Adamsville, Alabama","https://en.wikipedia.org/wiki/Adamsville,_Alabama",South (East South Central),5074296
2,Alabama,"Addison, Alabama","https://en.wikipedia.org/wiki/Addison,_Alabama",South (East South Central),5074296
3,Alabama,"Akron, Alabama","https://en.wikipedia.org/wiki/Akron,_Alabama",South (East South Central),5074296
4,Alabama,"Alabaster, Alabama","https://en.wikipedia.org/wiki/Alabaster,_Alabama",South (East South Central),5074296


In [52]:
state_city_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22157 entries, 0 to 22156
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   state              22157 non-null  object
 1   page_title         22157 non-null  object
 2   url                22157 non-null  object
 3   regional_division  22157 non-null  object
 4   Population (2022)  22157 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.0+ MB


### Defining some Constants for Easier Use*:

In [53]:
#########
#
#    CONSTANTS
#

# The basic English Wikipedia API endpoint
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"

# We'll assume that there needs to be some throttling for these requests - we should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': '<arjuns31@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

# This is just a list of English Wikipedia article titles that we can use for example requests
ARTICLE_TITLES = [ 'Bison', 'Northern flicker', 'Red squirrel', 'Chinook salmon', 'Horseshoe bat' ]

# This is a string of additional page properties that can be returned see the Info documentation for
# what can be included. If you don't want any this can simply be the empty string
PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"
#PAGEINFO_EXTENDED_PROPERTIES = ""

# This template lists the basic parameters for making this
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",           # to simplify this should be a single page title at a time
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}


### Making the API Call*:

In [54]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    
    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['titles'] = article_title

    if not request_template['titles']:
        raise Exception("Must supply an article title to make a pageinfo request.")

    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or any other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


In [ ]:
# Opened a json file to store the pageinfo data
with open('wiki_city_data.json', 'w', encoding='utf-8') as f:
    
    for i in range(len(cities)):
        print("Getting pageinfo data for: ",cities[i])
        info = request_pageinfo_per_article(cities[i])
        json.dump(info,f,indent=4)

Getting pageinfo data for:  Abbeville, Alabama
Getting pageinfo data for:  Adamsville, Alabama
Getting pageinfo data for:  Addison, Alabama
Getting pageinfo data for:  Akron, Alabama
Getting pageinfo data for:  Alabaster, Alabama
Getting pageinfo data for:  Albertville, Alabama
Getting pageinfo data for:  Alexander City, Alabama
Getting pageinfo data for:  Aliceville, Alabama
Getting pageinfo data for:  Allgood, Alabama
Getting pageinfo data for:  Altoona, Alabama
Getting pageinfo data for:  Andalusia, Alabama
Getting pageinfo data for:  Anderson, Lauderdale County, Alabama
Getting pageinfo data for:  Anniston, Alabama
Getting pageinfo data for:  Arab, Alabama
Getting pageinfo data for:  Ardmore, Alabama
Getting pageinfo data for:  Argo, Alabama
Getting pageinfo data for:  Ariton, Alabama
Getting pageinfo data for:  Arley, Alabama
Getting pageinfo data for:  Ashford, Alabama
Getting pageinfo data for:  Ashland, Alabama
Getting pageinfo data for:  Ashville, Alabama
Getting pageinfo data

Getting pageinfo data for:  Graysville, Alabama
Getting pageinfo data for:  Greensboro, Alabama
Getting pageinfo data for:  Greenville, Alabama
Getting pageinfo data for:  Grimes, Alabama
Getting pageinfo data for:  Grove Hill, Alabama
Getting pageinfo data for:  Guin, Alabama
Getting pageinfo data for:  Gulf Shores, Alabama
Getting pageinfo data for:  Guntersville, Alabama
Getting pageinfo data for:  Gurley, Alabama
Getting pageinfo data for:  Gu-Win, Alabama
Getting pageinfo data for:  Hackleburg, Alabama
Getting pageinfo data for:  Haleburg, Alabama
Getting pageinfo data for:  Haleyville, Alabama
Getting pageinfo data for:  Hamilton, Alabama
Getting pageinfo data for:  Hammondville, Alabama
Getting pageinfo data for:  Hanceville, Alabama
Getting pageinfo data for:  Harpersville, Alabama
Getting pageinfo data for:  Hartford, Alabama
Getting pageinfo data for:  Hartselle, Alabama
Getting pageinfo data for:  Hayden, Alabama
Getting pageinfo data for:  Hayneville, Alabama
Getting pagein

Getting pageinfo data for:  Red Bay, Alabama
Getting pageinfo data for:  Red Level, Alabama
Getting pageinfo data for:  Reece City, Alabama
Getting pageinfo data for:  Reform, Alabama
Getting pageinfo data for:  Rehobeth, Alabama
Getting pageinfo data for:  Repton, Alabama
Getting pageinfo data for:  Ridgeville, Alabama
Getting pageinfo data for:  River Falls, Alabama
Getting pageinfo data for:  Riverside, Alabama
Getting pageinfo data for:  Riverview, Alabama
Getting pageinfo data for:  Roanoke, Alabama
Getting pageinfo data for:  Robertsdale, Alabama
Getting pageinfo data for:  Rockford, Alabama
Getting pageinfo data for:  Rogersville, Alabama
Getting pageinfo data for:  Rosa, Alabama
Getting pageinfo data for:  Russellville, Alabama
Getting pageinfo data for:  Rutledge, Alabama
Getting pageinfo data for:  St. Florian, Alabama
Getting pageinfo data for:  Samson, Alabama
Getting pageinfo data for:  Sand Rock, Alabama
Getting pageinfo data for:  Sanford, Alabama
Getting pageinfo data f

Getting pageinfo data for:  Centreville, Alabama
Getting pageinfo data for:  Chatom, Alabama
Getting pageinfo data for:  Chelsea, Alabama
Getting pageinfo data for:  Cherokee, Alabama
Getting pageinfo data for:  Chickasaw, Alabama
Getting pageinfo data for:  Childersburg, Alabama
Getting pageinfo data for:  Citronelle, Alabama
Getting pageinfo data for:  Clanton, Alabama
Getting pageinfo data for:  Clay, Alabama
Getting pageinfo data for:  Clayhatchee, Alabama
Getting pageinfo data for:  Clayton, Alabama
Getting pageinfo data for:  Cleveland, Alabama
Getting pageinfo data for:  Clio, Alabama
Getting pageinfo data for:  Coaling, Alabama
Getting pageinfo data for:  Coffee Springs, Alabama
Getting pageinfo data for:  Coffeeville, Alabama
Getting pageinfo data for:  Coker, Alabama
Getting pageinfo data for:  Collinsville, Alabama
Getting pageinfo data for:  Colony, Alabama
Getting pageinfo data for:  Columbia, Alabama
Getting pageinfo data for:  Columbiana, Alabama
Getting pageinfo data fo

Getting pageinfo data for:  Lineville, Alabama
Getting pageinfo data for:  Lipscomb, Alabama
Getting pageinfo data for:  Lisman, Alabama
Getting pageinfo data for:  Littleville, Alabama
Getting pageinfo data for:  Livingston, Alabama
Getting pageinfo data for:  Loachapoka, Alabama
Getting pageinfo data for:  Lockhart, Alabama
Getting pageinfo data for:  Locust Fork, Alabama
Getting pageinfo data for:  Louisville, Alabama
Getting pageinfo data for:  Lowndesboro, Alabama
Getting pageinfo data for:  Loxley, Alabama
Getting pageinfo data for:  Luverne, Alabama
Getting pageinfo data for:  Lynn, Alabama
Getting pageinfo data for:  Madison, Alabama
Getting pageinfo data for:  Madrid, Alabama
Getting pageinfo data for:  Magnolia Springs, Alabama
Getting pageinfo data for:  Malvern, Alabama
Getting pageinfo data for:  Maplesville, Alabama
Getting pageinfo data for:  Margaret, Alabama
Getting pageinfo data for:  Marion, Alabama
Getting pageinfo data for:  Maytown, Alabama
Getting pageinfo data f

...

Getting pageinfo data for:  Great Barrington, Massachusetts
Getting pageinfo data for:  Greenfield, Massachusetts
Getting pageinfo data for:  Groton, Massachusetts
Getting pageinfo data for:  Groveland, Massachusetts
Getting pageinfo data for:  Hadley, Massachusetts
Getting pageinfo data for:  Halifax, Massachusetts
Getting pageinfo data for:  Hamilton, Massachusetts
Getting pageinfo data for:  Hampden, Massachusetts
Getting pageinfo data for:  Hancock, Massachusetts
Getting pageinfo data for:  Hanover, Massachusetts
Getting pageinfo data for:  Hanson, Massachusetts
Getting pageinfo data for:  Hardwick, Massachusetts
Getting pageinfo data for:  Harvard, Massachusetts
Getting pageinfo data for:  Harwich, Massachusetts
Getting pageinfo data for:  Hatfield, Massachusetts
Getting pageinfo data for:  Haverhill, Massachusetts
Getting pageinfo data for:  Hawley, Massachusetts
Getting pageinfo data for:  Heath, Massachusetts
Getting pageinfo data for:  Hingham, Massachusetts
Getting pageinfo d

Getting pageinfo data for:  Shutesbury, Massachusetts
Getting pageinfo data for:  Somerset, Massachusetts
Getting pageinfo data for:  Somerville, Massachusetts
Getting pageinfo data for:  South Hadley, Massachusetts
Getting pageinfo data for:  Southampton, Massachusetts
Getting pageinfo data for:  Southborough, Massachusetts
Getting pageinfo data for:  Southbridge, Massachusetts
Getting pageinfo data for:  Southwick, Massachusetts
Getting pageinfo data for:  Spencer, Massachusetts
Getting pageinfo data for:  Springfield, Massachusetts
Getting pageinfo data for:  Sterling, Massachusetts
Getting pageinfo data for:  Stockbridge, Massachusetts
Getting pageinfo data for:  Stoneham, Massachusetts
Getting pageinfo data for:  Stoughton, Massachusetts
Getting pageinfo data for:  Stow, Massachusetts
Getting pageinfo data for:  Sturbridge, Massachusetts
Getting pageinfo data for:  Sudbury, Massachusetts
Getting pageinfo data for:  Sunderland, Massachusetts
Getting pageinfo data for:  Sutton, Mass

Getting pageinfo data for:  Auburn, Michigan
Getting pageinfo data for:  Auburn Hills, Michigan
Getting pageinfo data for:  Au Gres, Michigan
Getting pageinfo data for:  Au Gres Township, Michigan
Getting pageinfo data for:  Augusta, Michigan
Getting pageinfo data for:  Augusta Charter Township, Michigan
Getting pageinfo data for:  Aurelius Township, Michigan
Getting pageinfo data for:  Au Sable Township, Iosco County, Michigan
Getting pageinfo data for:  Au Sable Township, Roscommon County, Michigan
Getting pageinfo data for:  Austin Township, Mecosta County, Michigan
Getting pageinfo data for:  Austin Township, Sanilac County, Michigan
Getting pageinfo data for:  Au Train Township, Michigan
Getting pageinfo data for:  Avery Township, Michigan
Getting pageinfo data for:  Backus Township, Michigan
Getting pageinfo data for:  Bad Axe, Michigan
Getting pageinfo data for:  Bagley Township, Michigan
Getting pageinfo data for:  Bainbridge Township, Michigan
Getting pageinfo data for:  Baldw

Getting pageinfo data for:  Brighton Township, Michigan
Getting pageinfo data for:  Briley Township, Michigan
Getting pageinfo data for:  Britton, Michigan
Getting pageinfo data for:  Brockway Township, Michigan
Getting pageinfo data for:  Bronson, Michigan
Getting pageinfo data for:  Bronson Township, Michigan
Getting pageinfo data for:  Brookfield Township, Eaton County, Michigan
Getting pageinfo data for:  Brookfield Township, Huron County, Michigan
Getting pageinfo data for:  Brooklyn, Michigan
Getting pageinfo data for:  Brooks Township, Michigan
Getting pageinfo data for:  Broomfield Township, Michigan
Getting pageinfo data for:  Brown Township, Michigan
Getting pageinfo data for:  Brown City, Michigan
Getting pageinfo data for:  Brownstown Charter Township, Michigan
Getting pageinfo data for:  Bruce Township, Chippewa County, Michigan
Getting pageinfo data for:  Bruce Township, Macomb County, Michigan
Getting pageinfo data for:  Buchanan, Michigan
Getting pageinfo data for:  Buc

Getting pageinfo data for:  Climax Township, Michigan
Getting pageinfo data for:  Clinton, Lenawee County, Michigan
Getting pageinfo data for:  Clinton Township, Lenawee County, Michigan
Getting pageinfo data for:  Clinton Charter Township, Michigan
Getting pageinfo data for:  Clinton Township, Oscoda County, Michigan
Getting pageinfo data for:  Clio, Michigan
Getting pageinfo data for:  Clyde Township, Allegan County, Michigan
Getting pageinfo data for:  Clyde Township, St. Clair County, Michigan
Getting pageinfo data for:  Coe Township, Michigan
Getting pageinfo data for:  Cohoctah Township, Michigan
Getting pageinfo data for:  Coldsprings Township, Michigan
Getting pageinfo data for:  Coldwater, Michigan
Getting pageinfo data for:  Coldwater Township, Branch County, Michigan
Getting pageinfo data for:  Coldwater Township, Isabella County, Michigan
Getting pageinfo data for:  Coleman, Michigan
Getting pageinfo data for:  Colfax Township, Benzie County, Michigan
Getting pageinfo data 

In [ ]:
# Opened wiki_city_data.json, modify it so as to ensure it follows json rules.
with open('wiki_city_data.json', 'r') as file:
    data = file.read()

# Split the data into individual JSON objects
data = data.replace('}{', '},{')
json_objects = data.split(',')

# Add "[" at the beginning and "]" at the end
valid_json_data = "[" + ",".join(json_objects) + "]"

# Parse the valid JSON data
parsed_data = json.loads(valid_json_data)

# Write the parsed data to a new file
with open('parsed_wiki_city_data.json', 'w') as file:
    json.dump(parsed_data, file, indent=4)

print("Finish")

In [ ]:
parsed_city_data = pd.read_json("parsed_wiki_city_data.json")

In [ ]:
parsed_city_data.head()

Notice that the query column contains the data we require in JSON format, and it appears quite messy. 
We require the 'title' and the 'lastrevid' data from the messy JSON. The cell below achieves that.

In [ ]:
lastrevids = [] # List to store the lastrevid
titles = [] # list to store the titles

# Below, we access the lastrevid and title information from the nested JSON data
for index, row in parsed_city_data.iterrows():
    query_result = row['query']
    pages = query_result.get('pages', {})
    lastrevid = None
    title = None
    
    for key, page in pages.items():
        lastrevid = page.get('lastrevid')
        title = page.get('title')
        if lastrevid is not None:
            lastrevids.append(lastrevid)
            titles.append(title)
            break

    if lastrevid is None:
        lastrevids.append(None)
        titles.append(None)


# Add 'lastrevid' and 'page_title' to the DataFrame
parsed_city_data['lastrevid'] = lastrevids
parsed_city_data['page_title'] = titles

parsed_city_data.head()

In [ ]:
parsed_city_data = parsed_city_data.drop(['batchcomplete','query'], axis = 1)

In [ ]:
parsed_city_data['lastrevid'] = parsed_city_data['lastrevid'].astype("object")

In [ ]:
final_states = pd.concat([state_city_data, parsed_city_data['lastrevid']], axis = 1)
final_states.head()

In [ ]:
final_states.rename(columns={'page_title': 'article_title'}, inplace=True)

In [ ]:
final_states.shape

In [ ]:
final_states.info()

In [ ]:
final_states.to_csv("Final_States_Data.csv",index = False)

### * The code snippet in the cell has been developed by Dr. David W. McDonald, as acknowledged above
